In [1]:
from multi_molecule import *
from multi_reaction import *
from brute_force import *
from ga import *
from simple_es import *
from ase.calculators.emt import *
from ase.calculators.lj import *

thermo.py:760 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from KlippensteinH2O2.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:774 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-database/input/thermo/groups...
transport.py:294 loadGroups INFO Loading transport group database from /Users/nathan/Code/RMG-database/input/transport/groups...
database.py:165 loadFamilies INFO Loading the user-specified kinetics families from /Users/nathan/Code/RMG-database/input/kinetics/famil

In [18]:
multi_object = Multi_Reaction("OCCCC+[O]O_OCC[CH]C+OO", "H_Abstraction")
multi_object.multi_ts.ase_ts.set_calculator(EMT())

reaction.py:300 generateBoundsMatrix INFO Generating bounds matrix for CCCCO.[O]O
reaction.py:338 bmPreEdit INFO Changing lower limit 2.9 to 2.419726
reaction.py:338 bmPreEdit INFO Changing lower limit 3.65 to 3.28879259416
reaction.py:338 bmPreEdit INFO Changing lower limit 2.515309 to 2.455244
reaction.py:338 bmPreEdit INFO Changing lower limit 3.65 to 3.28879259416
molecule.py:140 rd_embed INFO RDKit failed to embed on attempt 1 of 1000
molecule.py:140 rd_embed INFO RDKit failed to embed on attempt 2 of 1000
molecule.py:140 rd_embed INFO RDKit failed to embed on attempt 3 of 1000
molecule.py:140 rd_embed INFO RDKit failed to embed on attempt 4 of 1000
molecule.py:140 rd_embed INFO RDKit failed to embed on attempt 5 of 1000
molecule.py:140 rd_embed INFO RDKit failed to embed on attempt 6 of 1000
molecule.py:140 rd_embed INFO RDKit failed to embed on attempt 7 of 1000
molecule.py:140 rd_embed INFO RDKit failed to embed on attempt 8 of 1000
molecule.py:140 rd_embed INFO RDKit failed to

In [19]:
multi_object.multi_ts.torsions[0].left_mask

[True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False]

In [20]:
delta=30 
population_size=200
logging.info("Creating initial population of {} individuals from random guesses".format(population_size))

df = None

possible_dihedrals = np.arange(0, 360, delta)
population = []

possible_angles = np.arange(0, 180, delta)

logging.info("The object given is a `Multi_Reaction` object")

torsion_object = multi_object.multi_ts

for i in range(population_size):

    dihedrals = []
    for torsion in torsion_object.torsions:
        dihedral = np.random.choice(possible_dihedrals)
        dihedrals.append(dihedral)
        i, j, k, l = torsion.indices
        right_mask = torsion.right_mask

        torsion_object.ase_ts.set_dihedral(
            a1=i,
            a2=j,
            a3=k,
            a4=l,
            angle=float(dihedral),
            mask=right_mask
        )

    torsion_object.update_ts_from_ase_ts()

    e = torsion_object.ase_ts.get_potential_energy()

    population.append([e] + dihedrals)
    columns = ["Energy"]


logging.info("Creating a dataframe of the initial population")
df = pd.DataFrame(population)
for i in range(len(torsion_object.torsion_list)):
    columns = columns + ["Torsion " + str(i)]
df.columns = columns
df = df.sort_values("Energy")

<ipython-input-20-56054c89a253>:3 <module> INFO Creating initial population of 200 individuals from random guesses
<ipython-input-20-56054c89a253>:12 <module> INFO The object given is a `Multi_Reaction` object
<ipython-input-20-56054c89a253>:42 <module> INFO Creating a dataframe of the initial population


In [22]:
multi_object.multi_ts.view_ts()

In [23]:
from copy import deepcopy
df_test = deepcopy(df)
df_test

,Energy,Torsion 0,Torsion 1,Torsion 2,Torsion 3
62,7.534156,0,330,270,30
5,7.537622,30,0,240,180
84,7.577418,30,300,240,240
58,7.580867,30,0,180,300
36,7.601744,0,30,120,90
6,7.621881,30,0,270,120
74,7.663009,60,330,240,90
144,7.669903,30,30,240,150
37,7.679151,60,0,180,240
138,7.682343,0,330,150,30


In [24]:
top_percent=0.30
logging.info("Selecting the top population")
population_size = df_test.shape[0]
top_population = population_size * top_percent
top = df_test.iloc[:int(top_population), :]

<ipython-input-24-293f1f443c2d>:2 <module> INFO Selecting the top population


In [25]:
top

,Energy,Torsion 0,Torsion 1,Torsion 2,Torsion 3
62,7.534156,0,330,270,30
5,7.537622,30,0,240,180
84,7.577418,30,300,240,240
58,7.580867,30,0,180,300
36,7.601744,0,30,120,90
6,7.621881,30,0,270,120
74,7.663009,60,330,240,90
144,7.669903,30,30,240,150
37,7.679151,60,0,180,240
138,7.682343,0,330,150,30


In [26]:
tolerance=1e-4
max_generations=1
store_generations=False
store_directory="."

In [27]:
top_population = top.shape[0]

population_size = df_test.shape[0]

# Takes each of the molecule objects
if "Multi_Molecule" in str(multi_object.__class__):
    ase_object = multi_object.ase_molecule
    torsions = multi_object.torsions

elif "Multi_Reaction" in str(multi_object.__class__):
    ase_object = multi_object.multi_ts.ase_ts
    torsions = multi_object.multi_ts.torsions

elif "Multi_TS" in str(multi_object.__class__):
    ase_object = multi_object.ase_ts
    torsions = multi_object.torsions
    
columns, torsions, ase_object.get_calculator()

(['Energy', 'Torsion 0', 'Torsion 1', 'Torsion 2', 'Torsion 3'],
 [<geometry.Torsion instance at 0x1173a8320>,
  <geometry.Torsion instance at 0x1173a8518>,
  <geometry.Torsion instance at 0x1173a8758>,
  <geometry.Torsion instance at 0x1173a8f38>],
 <ase.calculators.emt.EMT instance at 0x1173418c0>)

In [29]:
#from copy import deepcopy
gen_number = 0
complete = False
while complete == False:
    gen_number += 1
    logging.info("Performing Simple ES on generation {}".format(gen_number))

    results = []
    for individual in range(population_size):
        dihedrals = []
        
        for index, torsion in enumerate(torsions):
            i, j, k, l = torsion.indices
            right_mask = torsion.right_mask

            dihedral = random.gauss(top.mean()[index + 1], top.std()[index + 1])
            dihedrals.append(dihedral)
            
            ase_object.set_dihedral(a1=i,
                                    a2=j,
                                    a3=k,
                                    a4=l,
                                    angle=float(dihedral),
                                    mask=right_mask)

            # Updating the molecule
            if "Multi_Molecule" in str(multi_object.__class__):
                multi_object.update_geometry_from_ase_mol()

            elif "Multi_Reaction" in str(multi_object.__class__):
                multi_object.multi_ts.update_ts_from_ase_ts()

            elif "Multi_TS" in str(multi_object.__class__):
                multi_object.update_ts_from_ase_ts()

        e = ase_object.get_potential_energy()
        results.append([e] + dihedrals)

    df = pd.DataFrame(results)
    logging.info("Creating the DataFrame of results for the {}th generation".format(gen_number))

    columns = ["Energy"]
    for i in range(len(torsions)):
        columns = columns + ["Torsion " + str(i)]

    df.columns = columns
    df = df.sort_values("Energy")

    if store_generations == True:
        # This portion stores each generation if desired

        if "Multi_Reaction" in str(multi_object.__class__):
            generation_name = "rxn_simple_es_generation_{}.pkl".format(gen_number)

        elif "Multi_Molecule" in str(multi_object.__class__):
            generation_name = "mol_simple_es_generation_{}.pkl".format(gen_number)

        elif "Multi_TS" in str(multi_object.__class__):
            generation_name = "ts_simple_es_generation_{}.pkl".format(gen_number)

        else:
            generation_name = "simple_es_generation_{}.pkl".format(gen_number)

        f = open(os.path.join(store_directory, generation_name), "w")
        pickle.dump(df, f)

    #top = df.iloc[:int(top_population), :]

    if gen_number >= max_generations:
        complete = True
        logging.info("Max generations reached. Simple ES complete.")
    if top.std()[0] / top.mean()[0] < tolerance:
        complete = True
        logging.info("Cutoff criteria reached. Simple ES complete.")

<ipython-input-29-b13b4861e516>:6 <module> INFO Performing Simple ES on generation 1
<ipython-input-29-b13b4861e516>:40 <module> INFO Creating the DataFrame of results for the 1th generation
<ipython-input-29-b13b4861e516>:71 <module> INFO Max generations reached. Simple ES complete.


In [30]:
ase_object.get_potential_energy()

7.728885776936803

In [31]:
df

,Energy,Torsion 0,Torsion 1,Torsion 2,Torsion 3
161,7.565808,346.814666,-18.583807,320.016508,168.120530
183,7.566381,18.022065,296.420516,208.221596,102.228833
148,7.645899,24.411081,313.287343,158.477544,179.436934
156,7.648359,22.078560,279.095032,307.847403,282.268034
40,7.650507,14.268985,43.269466,241.907607,83.621001
18,7.683885,292.349547,84.583304,210.460482,100.183214
90,7.684976,33.788939,-51.609159,144.395029,378.995049
176,7.688202,201.164574,62.516931,208.595387,369.009404
30,7.689666,222.056888,30.637510,230.134191,188.006168
37,7.691001,288.896615,48.802397,353.518073,106.395519
